# 匯入套件

In [1]:
import requests
import urllib
import json
from jsonpath import jsonpath
import jmespath
import pandas as pd
from datetime import datetime,timedelta
import time

# HTTP 請求參數與鏈接

In [2]:
def get_link_headers():
    keyword = '桌遊'
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36 Edg/88.0.705.68',
        'x-api-source': 'pc',
        'referer': f'https://shopee.tw/search?keyword={urllib.parse.quote(keyword)}'
    }
    
    s = requests.Session()
    url = 'https://shopee.tw/api/v4/search/product_labels'
    r = s.get(url, headers=headers)
    return s,headers

# 獲取api內容

In [3]:
def get_api_content(url):
    s,headers=get_link_headers()
    r = s.get(url, headers=headers)
    clean_needed = r.text
    json_data = json.loads('['+clean_needed+']')
    return json_data

# 商店部分

In [4]:
def shop():
    time=0
    for shoptime in df['商店api']:
        temp=jmespath.search('[].data.name',get_api_content(shoptime))
        df.loc[time,'店鋪名稱']=temp[0]
        time+=1
    return df

# 商品部分

In [5]:
def item():
    time=0
    for itemtime in df['商品api']:
        temp0=jmespath.search('[].item.models[].price',get_api_content(itemtime))
        df.loc[time,'選項售價']=",".join(str(x/100000) for x in temp0)
        temp1=jmespath.search('[].item.models[].sold',get_api_content(itemtime))
        df.loc[time,'選項銷量']=",".join(str(x) for x in temp1)
        time+=1
    return df

In [6]:
final_data = []
for time in range(1):
    time*=50
    url = f'https://shopee.tw/api/v2/search_items/?by=relevancy&keyword=桌遊&limit=50&newest={time}&order=desc&page_type=search&version=2'
    result=jmespath.search('[].items[*].{"店家帳號":shopid,"上架時間":ctime,"上架天數":ctime,"itemID":itemid,"商品標題":name,"商品網址":[name,shopid,itemid],"總銷售件數":historical_sold,"30天內銷售件數":sold,"售價":[price_min,price_max],"平均日銷(總銷件/上架天數)":[historical_sold,ctime],"喜歡數":liked_count,"消費潛力(總銷件/喜歡數)":[historical_sold,liked_count],"評價數":cmt_count,"評價綜合分數":item_rating.rating_star,"給評率(幾個評價/總銷件)":[cmt_count,historical_sold],"商品選項":tier_variations[0].options,"商品api":[itemid,shopid],"商店api":shopid}',get_api_content(url))
    final_data.append(result)
df = pd.DataFrame(final_data[0][0])

In [7]:
df['上架時間']=df['上架時間'].apply(lambda x:datetime.fromtimestamp(x))
df['上架天數']=df['上架天數'].apply(lambda x:(datetime.now()-datetime.fromtimestamp(x)).days)
df['商品網址']=df['商品網址'].apply(lambda x:"http://shopee.tw/"+str(x[0])+"-i."+str(x[1])+'.'+str(x[2]))
df['售價']=df['售價'].apply(lambda x:[int(a/100000) for a in x])
df['平均日銷(總銷件/上架天數)']=df['平均日銷(總銷件/上架天數)'].apply(lambda x:(x[0]/(datetime.now()-datetime.fromtimestamp(x[1])).days))
df['消費潛力(總銷件/喜歡數)']=df['消費潛力(總銷件/喜歡數)'].apply(lambda x:0 if x[1]==0 else (x[0]/x[1]))
df['給評率(幾個評價/總銷件)']=df['給評率(幾個評價/總銷件)'].apply(lambda x:0 if x[1]==0 else (x[0]/x[1]))
df['商品api']=df['商品api'].apply(lambda x:"https://shopee.tw/api/v2/item/get?itemid="+str(x[0])+"&shopid="+str(x[1]))
df['商店api']=df['商店api'].apply(lambda x:"https://shopee.tw/api/v4/product/get_shop_info?shopid="+str(x))
item()
shop()

,店家帳號,上架時間,上架天數,itemID,商品標題,商品網址,總銷售件數,30天內銷售件數,售價,平均日銷(總銷件/上架天數),...,消費潛力(總銷件/喜歡數),評價數,評價綜合分數,給評率(幾個評價/總銷件),商品選項,商品api,商店api,選項售價,選項銷量,店鋪名稱
0,285542537,2022-09-08 16:13:51,-544,6165266700,★現貨-快速出貨★西洋棋 國際象棋 實木高檔套裝 大號木質折疊棋盤 比賽專用chess 女王...,http://shopee.tw/★現貨-快速出貨★西洋棋 國際象棋 實木高檔套裝 大號木質...,28,4,"[1346, 2749]",-0.051471,...,0.500000,15,0.965763,0.535714,"[特大號45X45王高10.5c, 超大號39X39cm 送入門指, 特大45X45王高10...",https://shopee.tw/api/v2/item/get?itemid=61652...,https://shopee.tw/api/v4/product/get_shop_info...,"972.22393,1089.11196,1195.51143,997.92945,3200...","4,11,1,4,0,0,5,9",💕網拍首選💕極有家旗艦店 戶外 家居 3c生活館
1,265965505,2029-07-06 06:12:56,-3036,4437730450,台灣現貨秒發🚀 拉密 以色列拉密麻將 桌遊 旅行袋裝 盒裝版 大字版派對桌遊,http://shopee.tw/台灣現貨秒發🚀 拉密 以色列拉密麻將 桌遊 旅行袋裝 盒裝...,519,205,"[255, 1171]",-0.170949,...,1.206977,241,1.781058,0.464355,"[旅行, 旅行, , , , 狼人殺-, , , , 找]",https://shopee.tw/api/v2/item/get?itemid=44377...,https://shopee.tw/api/v4/product/get_shop_info...,"235.39015,652.3513,876.62614,866.92949,1299.82...","344,134,143,62,33,11,23,4,0,0",蝦拼流行百貨
2,25886973,2068-09-03 18:51:04,-17341,5479172260,🔥現貨🔥《遊慾.前戲篇》成人卡牌遊戲 情慾 多人性愛 遊戲 約炮 炮友 多人運動,http://shopee.tw/🔥現貨🔥《遊慾.前戲篇》成人卡牌遊戲 情慾 多人性愛 遊戲...,59,46,"[1108, 1518]",-0.003402,...,0.440299,22,5.278494,0.372881,[],https://shopee.tw/api/v2/item/get?itemid=54791...,https://shopee.tw/api/v4/product/get_shop_info...,1125.83945,70,嚇嚇情趣用品店(shotshotshop)
3,243296741,2054-01-19 15:28:06,-12000,7230544866,‼️台灣現貨‼️拉密 以色列拉密麻將 桌遊 旅行袋裝 盒裝,http://shopee.tw/‼️台灣現貨‼️拉密 以色列拉密麻將 桌遊 旅行袋裝 盒裝...,3785,256,"[257, 1722]",-0.315417,...,2.336420,1136,4.463234,0.300132,"[旅行, 旅行, , , 六人版（2, 狼人殺-, (加購)狼人殺2020, (加購)阿瓦,...",https://shopee.tw/api/v2/item/get?itemid=72305...,https://shopee.tw/api/v4/product/get_shop_info...,"233.96979,453.68743,713.30516,1315.71813,1497....","1501,917,452,185,218,55,10,4,15",Life樂活工坊
4,11767551,2059-01-04 18:24:56,-13811,6414226582,『旅遊便攜！買一送二』迷你麻將桌 迷你麻將 旅行麻將 袖珍麻將 旅行麻將 小型麻將 麻將桌 麻將桌,http://shopee.tw/『旅遊便攜！買一送二』迷你麻將桌 迷你麻將 旅行麻將 袖珍...,556,225,"[685, 746]",-0.040258,...,1.432990,233,1.586553,0.419065,"[迷你麻, 迷你麻, 迷你麻]",https://shopee.tw/api/v2/item/get?itemid=64142...,https://shopee.tw/api/v4/product/get_shop_info...,"945.88104,836.30706,570.83541","249,158,151",大爆炸3C-手機配件3C專賣店
5,80232902,2030-07-28 15:17:26,-3424,2314665741,[星桌遊]矮人礦坑 妙探尋兇 桌遊 富饒之城 抵抗組織 開膛手 花火 水瓶座 狼人 德國小強...,http://shopee.tw/[星桌遊]矮人礦坑 妙探尋兇 桌遊 富饒之城 抵抗組織 開...,1799,83,"[95, 241]",-0.525409,...,1.565709,496,2.940461,0.275709,"[經典 富饒之城（送, 黃盒, , 水瓶座, 情書一盒(超, , 超, 藍盒誰是牛頭, 矮人...",https://shopee.tw/api/v2/item/get?itemid=23146...,https://shopee.tw/api/v4/product/get_shop_info...,"186.97811,153.13653,91.08298,109.01452,195.650...","204,101,59,115,154,247,69,110,141,26,69,45,140...",星桌遊阿瓦隆矮人金礦誰是臥底卡卡頌富饒之城大富翁狼人殺三國殺小黑屋犯罪現場
6,59509357,2036-02-06 17:10:20,-5443,6031176364,【24H寄出台灣現貨99免運】暑假桌遊 害你在心口難開 爆笑不要做挑戰遊戲卡牌 遊戲 桌遊 ...,http://shopee.tw/【24H寄出台灣現貨99免運】暑假桌遊 害你在心口難開 爆...,9067,356,"[123, 158]",-1.665809,...,3.737428,1952,1.358874,0.215286,"[基礎版綠盒(贈頭帶*8、空白卡, 擴充版黃盒(可獨立玩 贈頭帶, 🐺狼人]",https://shopee.tw/api/v2/item/get?itemid=60311...,https://shopee.tw/api/v4/product/get_shop_info...,"137.4034,144.25476,119.28907","4193,5556,108",七七市集 - 好物嚴選 ； 買東西找七七
7,25886973,2028-04-27 08:53:35,-2601,5479172260,🔥現貨🔥《遊慾.前戲篇》成人卡牌遊戲 情慾 多人性愛 遊戲 約炮 炮友 多人運動,http://shopee.tw/🔥現貨🔥《遊慾.前戲篇》成人卡牌遊戲 情慾 多人性愛 遊戲...,61,28,"[1683, 1308]",-0.023453,...,0.508333,23,0.896510,0.377049,[],https://shopee.tw/api/v2/item/get?itemid=54791...,https://shopee.tw/api/v4/product/get_shop_info...,1592.94782,65,嚇嚇情趣用品店(shotshotshop)
8,75612907,2058-01-20 08:15:55,-13461,2380890861,現貨阿瓦隆害你在心口難開化妝舞會你咬我啊大富翁富饒之城政變花火水瓶座 誰是牛頭王一夜終極狼人...,http://shopee.tw/現貨阿瓦隆害你在心口難開化妝舞會你咬我啊大富翁富饒之城政變...,5235,21,"[89, 470]",-0.388901,...,7.331933,851,1.579528,0.162560,"[阿瓦隆 繁體 送蘭羅斯特擴充2, 政变 中英文（20, 抵抗, 驚濤駭浪（怒海求, , ...",https://shopee.tw/api/v2/item/get?itemid=23808...,https://shopee.tw/api/v4/product/get_shop_info...,"124.22541,186.1755,150.78111,168.27388,152.119...","832,187,48,89,160,103,649,122,365,551,394,82,1...",黑匣子-遊戲人生
9,28893170,2053-06-08 07:36:54,-1

In [8]:
for time in range(len(df)):
    df.loc[time,'售價']=",".join(str(x) for x in df.loc[time,'售價'])
    df.loc[time,'商品選項']=",".join(x for x in df.loc[time,'商品選項'])

In [9]:
today = datetime.today().strftime("%Y%m%d")
df.to_csv(f"{today}.csv",index=False)

In [10]:
df

,店家帳號,上架時間,上架天數,itemID,商品標題,商品網址,總銷售件數,30天內銷售件數,售價,平均日銷(總銷件/上架天數),...,消費潛力(總銷件/喜歡數),評價數,評價綜合分數,給評率(幾個評價/總銷件),商品選項,商品api,商店api,選項售價,選項銷量,店鋪名稱
0,285542537,2022-09-08 16:13:51,-544,6165266700,★現貨-快速出貨★西洋棋 國際象棋 實木高檔套裝 大號木質折疊棋盤 比賽專用chess 女王...,http://shopee.tw/★現貨-快速出貨★西洋棋 國際象棋 實木高檔套裝 大號木質...,28,4,"1346,2749",-0.051471,...,0.500000,15,0.965763,0.535714,"特大號45X45王高10.5c,超大號39X39cm 送入門指,特大45X45王高10.5+...",https://shopee.tw/api/v2/item/get?itemid=61652...,https://shopee.tw/api/v4/product/get_shop_info...,"972.22393,1089.11196,1195.51143,997.92945,3200...","4,11,1,4,0,0,5,9",💕網拍首選💕極有家旗艦店 戶外 家居 3c生活館
1,265965505,2029-07-06 06:12:56,-3036,4437730450,台灣現貨秒發🚀 拉密 以色列拉密麻將 桌遊 旅行袋裝 盒裝版 大字版派對桌遊,http://shopee.tw/台灣現貨秒發🚀 拉密 以色列拉密麻將 桌遊 旅行袋裝 盒裝...,519,205,"255,1171",-0.170949,...,1.206977,241,1.781058,0.464355,"旅行,旅行,,,,狼人殺-,,,,找",https://shopee.tw/api/v2/item/get?itemid=44377...,https://shopee.tw/api/v4/product/get_shop_info...,"235.39015,652.3513,876.62614,866.92949,1299.82...","344,134,143,62,33,11,23,4,0,0",蝦拼流行百貨
2,25886973,2068-09-03 18:51:04,-17341,5479172260,🔥現貨🔥《遊慾.前戲篇》成人卡牌遊戲 情慾 多人性愛 遊戲 約炮 炮友 多人運動,http://shopee.tw/🔥現貨🔥《遊慾.前戲篇》成人卡牌遊戲 情慾 多人性愛 遊戲...,59,46,"1108,1518",-0.003402,...,0.440299,22,5.278494,0.372881,,https://shopee.tw/api/v2/item/get?itemid=54791...,https://shopee.tw/api/v4/product/get_shop_info...,1125.83945,70,嚇嚇情趣用品店(shotshotshop)
3,243296741,2054-01-19 15:28:06,-12000,7230544866,‼️台灣現貨‼️拉密 以色列拉密麻將 桌遊 旅行袋裝 盒裝,http://shopee.tw/‼️台灣現貨‼️拉密 以色列拉密麻將 桌遊 旅行袋裝 盒裝...,3785,256,"257,1722",-0.315417,...,2.336420,1136,4.463234,0.300132,"旅行,旅行,,,六人版（2,狼人殺-,(加購)狼人殺2020,(加購)阿瓦,(加購)德",https://shopee.tw/api/v2/item/get?itemid=72305...,https://shopee.tw/api/v4/product/get_shop_info...,"233.96979,453.68743,713.30516,1315.71813,1497....","1501,917,452,185,218,55,10,4,15",Life樂活工坊
4,11767551,2059-01-04 18:24:56,-13811,6414226582,『旅遊便攜！買一送二』迷你麻將桌 迷你麻將 旅行麻將 袖珍麻將 旅行麻將 小型麻將 麻將桌 麻將桌,http://shopee.tw/『旅遊便攜！買一送二』迷你麻將桌 迷你麻將 旅行麻將 袖珍...,556,225,"685,746",-0.040258,...,1.432990,233,1.586553,0.419065,"迷你麻,迷你麻,迷你麻",https://shopee.tw/api/v2/item/get?itemid=64142...,https://shopee.tw/api/v4/product/get_shop_info...,"945.88104,836.30706,570.83541","249,158,151",大爆炸3C-手機配件3C專賣店
5,80232902,2030-07-28 15:17:26,-3424,2314665741,[星桌遊]矮人礦坑 妙探尋兇 桌遊 富饒之城 抵抗組織 開膛手 花火 水瓶座 狼人 德國小強...,http://shopee.tw/[星桌遊]矮人礦坑 妙探尋兇 桌遊 富饒之城 抵抗組織 開...,1799,83,"95,241",-0.525409,...,1.565709,496,2.940461,0.275709,"經典 富饒之城（送,黃盒,,水瓶座,情書一盒(超,,超,藍盒誰是牛頭,矮人礦坑1+2大盒版（...",https://shopee.tw/api/v2/item/get?itemid=23146...,https://shopee.tw/api/v4/product/get_shop_info...,"186.97811,153.13653,91.08298,109.01452,195.650...","204,101,59,115,154,247,69,110,141,26,69,45,140...",星桌遊阿瓦隆矮人金礦誰是臥底卡卡頌富饒之城大富翁狼人殺三國殺小黑屋犯罪現場
6,59509357,2036-02-06 17:10:20,-5443,6031176364,【24H寄出台灣現貨99免運】暑假桌遊 害你在心口難開 爆笑不要做挑戰遊戲卡牌 遊戲 桌遊 ...,http://shopee.tw/【24H寄出台灣現貨99免運】暑假桌遊 害你在心口難開 爆...,9067,356,"123,158",-1.665809,...,3.737428,1952,1.358874,0.215286,"基礎版綠盒(贈頭帶*8、空白卡,擴充版黃盒(可獨立玩 贈頭帶,🐺狼人",https://shopee.tw/api/v2/item/get?itemid=60311...,https://shopee.tw/api/v4/product/get_shop_info...,"137.4034,144.25476,119.28907","4193,5556,108",七七市集 - 好物嚴選 ； 買東西找七七
7,25886973,2028-04-27 08:53:35,-2601,5479172260,🔥現貨🔥《遊慾.前戲篇》成人卡牌遊戲 情慾 多人性愛 遊戲 約炮 炮友 多人運動,http://shopee.tw/🔥現貨🔥《遊慾.前戲篇》成人卡牌遊戲 情慾 多人性愛 遊戲...,61,28,"1683,1308",-0.023453,...,0.508333,23,0.896510,0.377049,,https://shopee.tw/api/v2/item/get?itemid=54791...,https://shopee.tw/api/v4/product/get_shop_info...,1592.94782,65,嚇嚇情趣用品店(shotshotshop)
8,75612907,2058-01-20 08:15:55,-13461,2380890861,現貨阿瓦隆害你在心口難開化妝舞會你咬我啊大富翁富饒之城政變花火水瓶座 誰是牛頭王一夜終極狼人...,http://shopee.tw/現貨阿瓦隆害你在心口難開化妝舞會你咬我啊大富翁富饒之城政變...,5235,21,"89,470",-0.388901,...,7.331933,851,1.579528,0.162560,"阿瓦隆 繁體 送蘭羅斯特擴充2,政变 中英文（20,抵抗,驚濤駭浪（怒海求,,偉人猜拳,黃...",https://shopee.tw/api/v2/item/get?itemid=23808...,https://shopee.tw/api/v4/product/get_shop_info...,"124.22541,186.1755,150.78111,168.27388,152.119...","832,187,48,89,160,103,649,122,365,551,394,82,1...",黑匣子-遊戲人生
9,28893170,2053-06-08 07:36:54,-11774,8813163592,神吐槽【瘋桌遊】過年派對必備桌遊│party、桌遊、歡樂、想像力│繁體中文唯,h